# Memento

## O que é?

O padrao _Memento_, prevê a utilização de uma classe _memento_ para guardar o estado de uma outra classe _originator_.

## Por quê?

Em certas aplicações é interessante armazenar o estado de um objeto para que esse seja restaurado no futuro. Contudo, ter conhecimento do estado interno de um objeto tende a quebrar o princípio de encapsulamento. O padrão memento resolve esse problema através de uma classe _memento_ que armazena o estado de outra sem expor os detalhes ao resto da aplicação.

## Estrutura: 

![struct](https://upload.wikimedia.org/wikipedia/commons/3/38/W3sDesign_Memento_Design_Pattern_UML.jpg)

## Exemplo:

O novo jogo da série _Fire Emble_ adicionou um mecânica chamada _Divine Pulse_, a qual permite que o jogador reverta as ações tomadas.

Vejamos um implementação simplificada de tal mecânica.

In [28]:
from collections import defaultdict

class Controller:
    """
    Manipulate characters
    """
    def __init__(self):
        self._history = defaultdict(list)
    
    def save_state(self, char):
        """Creates a snapshot of the character's state."""
        self._history[char].append({
            "hp": char._hp, "x": char._x, "y": char._y
        })
    
    def move(self, char, x, y):
        """Move a character."""
        self.save_state(char)
        char.move(x, y)
        print(f"Character {char} has moved {x} spaces to the right and {y} spaces up.")
        
    def damage(self, char, dmg):
        """Damage a character"""
        self.save_state(char)
        char.damage(dmg)
        print(f"Character {char} took {dmg} pts of damage.")
        
    def rewind(self, char):
        """Undo the character's most recent action."""
        previous_state = self._history[char].pop()
        char._hp = previous_state["hp"]
        char._x = previous_state["x"]
        char._y = previous_state["y"]
        

class Character:
    """
    Holds all information regarding a character.
    """
    
    def __init__(self, name, hp, x, y):
        self.name = name
        self._hp = hp
        self._x = x
        self._y = y
        
    def move(self, delta_x, delta_y):
        """Move the character by a given delta x and y."""
        self._x += delta_x
        self._y += delta_y
        
    def damage(self, damage):
        """Update HP so it takes into account the damage taken."""
        self._hp -= damage
        
    def __repr__(self):
        return f"{self.name}(hp: {self._hp}, position: ({self._x}, {self._y})"

In [29]:
ctl = Controller()
marth = Character("Marth", hp=20, x=0, y=0)
marth

Marth(hp: 20, position: (0, 0)

In [30]:
ctl.move(marth, 1, -1)

Character Marth(hp: 20, position: (1, -1) has moved 1 spaces to the right and -1 spaces up.


In [31]:
ctl.damage(marth, 10)

Character Marth(hp: 10, position: (1, -1) took 10 pts of damage.


Hora de desfazer a nossa ação.

In [32]:
ctl.rewind(marth)
marth

Marth(hp: 20, position: (1, -1)

Novamente...

In [33]:
ctl.rewind(marth)
marth

Marth(hp: 20, position: (0, 0)

O problema dessa implementação é que o _Controller_ precisa ter conhecimento do estado interno do _Character_. Isso so torna um problema quando novas classes são adicionadas, estados são modificados ou múltiplos controlers são empregados.

Modifiquemos adicionemos o atribute MP ao _Character_ e uma nova classe chamada _HQ_.

In [79]:
class Controller:
    """
    Manipulate characters
    """
    def __init__(self):
        self._history = defaultdict(list)
    
    def save_state(self, actor):
        """Creates a snapshot of the character's state."""
        if isinstance(actor, Character):
            self._history[actor].append({  # novo if statement
                "hp": actor._hp, "x": actor._x, "y": actor._y, "mp": actor._mp  # novo atributo
            })
        else:
            self._history[actor].append({
                "held_by": actor._held_by  # mais lógica
            })
    
    def move(self, char, x, y):
        """Move a character."""
        self.save_state(char)
        char.move(x, y)
        print(f"Character {char} has moved {x} spaces to the right and {y} spaces up.")
        
    def damage(self, char, dmg):
        """Damage a character"""
        self.save_state(char)
        char.damage(dmg)
        print(f"Character {char} took {dmg} pts of damage.")
        
    def capture(self, char, hq):
        """Character captures an HQ."""
        self.save_state(hq)
        hq.conquered_by(char.name)
        print(f"{hq} conquered by {char}")
        
    def rewind(self, actor):
        """Undo the character's most recent action."""
        
        if not self._history[actor]:
            print("Cannot go back any further.")
            return
        
        previous_state = self._history[actor].pop()
        
        if isinstance(actor, Character):
            actor._hp = previous_state["hp"]
            actor._x = previous_state["x"]
            actor._y = previous_state["y"]
            actor._mp = previous_state["mp"]  # mais uma linha
        else:
            actor._held_by = previous_state["held_by"]
            
class HQ:
    """
    Headquarters.
    """
    def __init__(self, name, held_by=""):
        self.name = name
        self._held_by = held_by
        
    def conquered_by(self, name):
        """HQ taken by some character."""
        self._held_by = name
        
    def __repr__(self):
        return f"{self.name}(held_by: {self._held_by})"
            
class Character:
    """
    Holds all information regarding a character.
    """
    
    def __init__(self, name, hp, mp, x, y):
        self.name = name
        self._hp = hp
        self._x = x
        self._y = y
        self._mp = mp
        
    def move(self, delta_x, delta_y):
        """Move the character by a given delta x and y."""
        self._x += delta_x
        self._y += delta_y
        
    def damage(self, damage):
        """Update HP so it takes into account the damage taken."""
        self._hp -= damage
        
    def __repr__(self):
        return f"{self.name}(hp: {self._hp}, mp: {self._mp}, position: ({self._x}, {self._y})"

In [71]:
ctl = Controller()
marth = Character("Marth", hp=20, mp=10, x=0, y=0)
marth

Marth(hp: 20, mp: 10, position: (0, 0)

In [72]:
ctl.move(marth, 1, -1)

Character Marth(hp: 20, mp: 10, position: (1, -1) has moved 1 spaces to the right and -1 spaces up.


In [73]:
ctl.damage(marth, 10)

Character Marth(hp: 10, mp: 10, position: (1, -1) took 10 pts of damage.


Hora de desfazer a nossa ação.

In [74]:
ctl.rewind(marth)
marth

Marth(hp: 20, mp: 10, position: (1, -1)

Novamente...

In [75]:
ctl.rewind(marth)
marth

Marth(hp: 20, mp: 10, position: (0, 0)

In [76]:
enemy_hq = HQ("Dark Castle", held_by="The Evil!")
enemy_hq

Dark Castle(held_by: The Evil!)

In [77]:
ctl.capture(marth, enemy_hq)

Dark Castle(held_by: Marth) conquered by Marth(hp: 20, mp: 10, position: (0, 0)


In [78]:
ctl.rewind(enemy_hq)
enemy_hq

Dark Castle(held_by: The Evil!)

Utilizando mementos esse problema é mitigado.

In [109]:
import abc


class Memento(metaclass=abc.ABCMeta):
    """
    Class used to store the internal state of a given class.
    """
    
    def __init__(self):
        self._state = None
    
    def set_state(self, state):
        """Store the state of an actor."""
        self._state = state
    
    def get_state(self):
        """Retrive the state of a given actor."""
        return self._state
    
    
class Actor(metaclass=abc.ABCMeta):
    """
    Class which we want to store the and restore the state of.
    """
    
    @abc.abstractmethod
    def create_memento(self):
        """Create the correct object."""
        pass
    
    @abc.abstractmethod
    def set_memento(self, memento):
        """Restore a previous state using a memento."""
        pass

    
class HQ(Actor):
    """
    Headquarters.
    """
    def __init__(self, name, held_by=""):
        self.name = name
        self._held_by = held_by
        
    def conquered_by(self, name):
        """HQ taken by some character."""
        self._held_by = name
        
    def create_memento(self):
        memento = HQMemento()
        memento.set_state({"held_by": self._held_by})
        return memento
    
    def set_memento(self, memento):
        state = memento.get_state()
        self._held_by = state["held_by"]
        
    def __repr__(self):
        return f"{self.name}(held_by: {self._held_by})"
    
    
class HQMemento(Memento):
    pass
    
            
class Character:
    """
    Holds all information regarding a character.
    """
    
    def __init__(self, name, hp, mp, x, y):
        self.name = name
        self._hp = hp
        self._x = x
        self._y = y
        self._mp = mp
        
    def move(self, delta_x, delta_y):
        """Move the character by a given delta x and y."""
        self._x += delta_x
        self._y += delta_y
        
    def damage(self, damage):
        """Update HP so it takes into account the damage taken."""
        self._hp -= damage
         
    def create_memento(self):
        memento = CharMemento()
        memento.set_state({"hp": self._hp, "x": self._x, "y": self._y, "mp": self._mp})
        return memento
    
    def set_memento(self, memento):
        state = memento.get_state()
        self._hp = state["hp"]
        self._x = state["x"]
        self._y = state["y"]
        self._mp = state["mp"]
        
    def __repr__(self):
        return f"{self.name}(hp: {self._hp}, mp: {self._mp}, position: ({self._x}, {self._y})"
    

class CharMemento(Memento):
    pass
    
    
class Controller:
    """
    Manipulate characters
    """
    def __init__(self):
        self._history = defaultdict(list)
    
    def save_state(self, actor):
        """Creates a snapshot of the character's state."""
        self._history[actor].append(actor.create_memento())  # lógica muito mais simples
        
    def move(self, char, x, y):
        """Move a character."""
        self.save_state(char)
        char.move(x, y)
        print(f"Character {char} has moved {x} spaces to the right and {y} spaces up.")
        
    def damage(self, char, dmg):
        """Damage a character"""
        self.save_state(char)
        char.damage(dmg)
        print(f"Character {char} took {dmg} pts of damage.")
        
    def capture(self, char, hq):
        """Character captures an HQ."""
        self.save_state(hq)
        hq.conquered_by(char.name)
        print(f"{hq} conquered by {char}")
        
    def rewind(self, actor):
        """Undo the character's most recent action."""
        
        if not self._history[actor]:
            print("Cannot go back any further.")
            return
        
        memento = self._history[actor].pop()
        actor.set_memento(memento)

In [110]:
ctl = Controller()
marth = Character("Marth", hp=20, mp=10, x=0, y=0)
marth

Marth(hp: 20, mp: 10, position: (0, 0)

In [111]:
ctl.move(marth, 1, -1)

Character Marth(hp: 20, mp: 10, position: (1, -1) has moved 1 spaces to the right and -1 spaces up.


In [112]:
ctl.damage(marth, 10)

Character Marth(hp: 10, mp: 10, position: (1, -1) took 10 pts of damage.


Hora de desfazer a nossa ação.

In [113]:
ctl.rewind(marth)
marth

Marth(hp: 20, mp: 10, position: (1, -1)

Novamente...

In [114]:
ctl.rewind(marth)
marth

Marth(hp: 20, mp: 10, position: (0, 0)

In [115]:
enemy_hq = HQ("Dark Castle", held_by="The Evil!")
enemy_hq

Dark Castle(held_by: The Evil!)

In [116]:
ctl.capture(marth, enemy_hq)

Dark Castle(held_by: Marth) conquered by Marth(hp: 20, mp: 10, position: (0, 0)


In [117]:
ctl.rewind(enemy_hq)
enemy_hq

Dark Castle(held_by: The Evil!)

## Prós e contras:

### Prós

- Permite que o encapsulamento das classes cujos estados internos queremos restaurar seja mantido.
- Como os estado do objeto não é mantido pelo objeto em si, é possível que este seja restaurado mesmo que a instância seja deletada.
- Os objetos a serem restaurados não precisam tomar conhecimento da lógica de negócio.

### Contras

- É necessária a criação de classes extras.
- O estado interno dos objetos deve ser relativamento pequeno para que evitar o uso excessivo de memória.
- A criação dos _mementos_ deve ser rápida ou o desempenho da aplicação pode ser afetado.


## Discussao:

1. The authors write that the "Caretaker" participant never operates on or examines the contents of a memento. Can you consider a case where a Caretaker would infact need to know the identity of a memento and thus need the ability to examine or query the contents of that memento? Would this break something in the pattern?